In [33]:
type Size =
    | Large
    | Medium
    | Small
    

type Lives =
    | Three
    | Two
    | One
    | Zero
    
type Ship = {
        Pos: float*float
        Vel: float*float
        Ang: float
        Lives: Lives
        Size: float
    }

type Asteroid = {
    Pos: float*float
    Vel: float*float
    Size: Size
    }

type Bullet = {
    Pos: float*float
    Ang: float
    Life: float
}

type Saucer = {
    Ypos: float
    Dir: bool
}

type Rotation =
    | Positive
    | Negative
    | None

type Input = {
    Thrust: bool
    Shoot: bool
    Rot: Rotation
}

type Game = {
    Ship: Ship
    Asteroids: Asteroid list
    Bullets: Bullet list
    Saucer: Saucer
    Score: int
}

let shipMaxVel = 1
let maxBullLife = 1
let fps = 1
let aspect_ratio = 1.5



In [ ]:
// check collision between pair of objects
let checkColAst_Bull (ast: Ast) (bull: Bullet) : bool =

let checkColAst_Ship (ast: Ast) (ship: Ship) : bool =

let checkColShip_Bull (ship: Ship) (bull: Bullet) : bool =

// kill Asts if collision
let updateAst (game: Game) =

let updateShip (game: Game) (input: Input) =

let updateBull (game: Game) (input: Input) =


let moveAst (ast: Ast) =

let moveShip (ship: Ship) (input: Input) =

let moveBull (bull: Bullet) =


let spawnAst =
    

let spawnShip =

In [2]:
let sizeMap = Map [(Size.Small, 0.01); (Size.Medium, 0.02); (Size.Large, 0.04)]

let checkCollisionShipAsteroid (ship: Ship, asteroid: Asteroid) = 
    // Devuelve verdadero si hay superposición entre la nave y un asteroide
    let distance = Math.Sqrt(Math.Pow((fst asteroid.Pos - fst ship.Pos),2) + Math.Pow((snd asteroid.Pos - snd ship.Pos),2))
    let radiiSum = sizeMap[asteroid.Size] + ship.Size
    (distance - radiiSum) < 0

let checkCollisionShipBullet (ship: Ship, bullet: Bullet) =
    // Devuelve verdadero si hay superposición entre la nave y una bala
    let distance = Math.Sqrt(Math.Pow((fst bullet.Pos - fst ship.Pos),2) + Math.Pow((snd bullet.Pos - snd ship.Pos),2))
    (distance - ship.Size) < 0 //la bala se supone puntual



In [17]:
let bulletsVelocity = 0.1

let normalizeValue (x: float, modulo: float) =
    // Esta función hace que los objetos que traspasan un borde la pantalla regresen por otro
    let rec normalize (x: float, modulo: float) =
        if x < 0.0 then
            normalize (x + modulo, modulo)
        else
            x % modulo
    normalize (x, modulo)

let moveBullet (bullet: Bullet) = 
    let Vx = -bulletsVelocity*Math.Cos(bullet.Ang)
    let Vy = -bulletsVelocity*Math.Sin(bullet.Ang)

    let newPosX = normalizeValue((fst bullet.Pos) + Vx/(float fps), aspect_ratio)
    let newPosY = normalizeValue((snd bullet.Pos) + Vy/(float fps), 1.0)
    let newPosition = (newPosX, newPosY)
    
    let newLife = bullet.Life + bulletsVelocity/(float fps)

    let newBullet = 
        { bullet with
            Pos = newPosition
            Life = newLife
        }
    newBullet

let moveAndClearBullets (bullets: List<Bullet>) = 
    bullets
    |> List.map (fun b -> moveBullet(b)) // Actualiza la posición de las balas y contabiliza su recorrido
    |> List.filter (fun b -> b.Life <= maxBullLife) // Elimina las balas cuyo recorrido supera el máximo establecido


In [22]:
let moveAsteroid (asteroid: Asteroid) =
    // Función que actualiza la posición de los asteroides
    // A los módulos se les adiciona el tamaño del objeto para que la reaparicion ocurra cuando el objeto deja de visualizarse en la pantalla
    let newPosX = normalizeValue((fst asteroid.Pos) + (fst asteroid.Vel)/(float fps), aspect_ratio + sizeMap[asteroid.Size])
    let newPosY = normalizeValue((snd asteroid.Pos) + (snd asteroid.Vel)/(float fps), 1.0 + sizeMap[asteroid.Size])
    let newPosition = (newPosX, newPosY)

    let newAsteroid = 
        { asteroid with
            Pos = newPosition
        }
    newAsteroid

In [ ]:
let spawnShip game: Game =
    // Reestablece el estado de la nave en el juego
    let newShip =
        { game.Ship with
            Pos = (0.5, aspect_ratio/2.0)
            Ang = 0.0
            Vel = (0.0, 0.0)
        }
    let newGame = 
        { game with
            Ship = newShip
        }
    newGame


In [27]:
let shootBullet (ship: Ship, bullets: List<Bullet>, input: Input) = 
    // Agrega una bala a la lista de balas cuando se ejecuta la acción de disparar
    let fire (ship: Ship, bullets: List<Bullet>) =
        let newBullet = 
            {
                Pos = (fst ship.Pos - ship.Size*Math.Cos(ship.Ang),
                    snd ship.Pos - ship.Size*Math.Sin(ship.Ang))
                Ang = ship.Ang
                Life = 0.0
            }
        bullets
        |> List.append [newBullet]
    
    match input.Shoot with
    | true -> fire (ship, bullets)
    | false -> bullets


In [26]:
let rotateShip (ship: Ship, input: Input) = 
    // Modifica la orientación de la nave en 5° si el input así lo indica
    let newAngle (rotation: Rotation) =
        match rotation with
        | Positive -> 5.0 * Math.PI/180.0
        | Negative -> -5.0 * Math.PI/180.0
        | None -> 0.0

    let newShip =
        { ship with
            Ang = newAngle(input.Rot)
        }
    newShip

In [44]:
let shipAcc = 1.0/(float fps)

let cartesianToPolar (x: float, y: float) =
    let r = Math.Sqrt(x**2 + y**2)
    let theta = Math.PI/2.0 - Math.Atan2 (y, x) // REVISAR!
    r, theta


// Funciones no finalizadas!
let capVelocity(ship: Ship) = 
    //

let accelerateShip (ship: Ship, input: Input) =
    let velocityDelta (ship: Ship, thrust: bool) =
        let (velR, velTheta) = cartesianToPolar(fst ship.Vel, snd ship.Vel)
        match thrust with
        | true -> (-shipAcc*Math.Cos(ship.Ang), -shipAcc*Math.Sin(ship.Ang))  // Aceleración con orientación al ángulo actual
        | false -> (+shipAcc*Math.Cos(velTheta), +shipAcc*Math.Sin(velTheta)) // Desaceleración con orientación a la velocidad actual
    
    let velDelta = velocityDelta(ship, thrust)

    let newVelocity =
        match cartesianToPolar(velDelta) with
        | (r, theta) when r > 0 && r < shipMaxVel -> (fst ship.Vel + fst velDelta, snd ship.Vel + snd velDelta)
        | (r, theta) when r <= 0  -> (0.0, 0.0)
        | (r, theta) when r >= shipMaxVel  -> (fst ship.Vel + fst velDelta, snd ship.Vel + snd velDelta)

    let newShip =
        { ship with
            Vel = (fst ship.Vel + fst velPlus, snd ship.Vel + snd velPlus)
        } 
        
    newShip

Stopped due to error


Error: input.fsx (17,24)-(17,31) typecheck error The type 'Ship' does not define the field, constructor or member 'Velship'. Maybe you want one of the following:
   Vel